## Dependency

In [32]:
import os
import yaml #pip install pyYAML
from utils.utils import convert_params
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import os
import neptune
import joblib

In [21]:
run = neptune.init_run(
    project="test-text/bootcamp-test",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5ODAzYzk0NS1iNjFiLTQ3ZDAtYjJlZS1hMDg2NTA1ZTliODgifQ==",
)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/test-text/bootcamp-test/e/BOOT-1


## path of data

In [22]:

data_path =  '..\Bootcamp_Machine_Learning_Production_Data_Preparation\dataset'

def load_pickle(child_path): #load data.pickle 
    data = pickle.load(open(os.path.join(data_path, child_path), 'rb'))
    return data

## load dataset

In [23]:
train_data = load_pickle("train_data.pickle")
test_data = load_pickle("test_data.pickle")

train_x = train_data['x']
train_y = train_data['y']
test_x = test_data['x']
test_y = test_data['y']

## load yaml

In [25]:
# Load methods config and model version from YAML
with open("configs/config.yaml", "r") as file:
    config = yaml.safe_load(file)

model_version = config["model_versions"]  # Get model version from YAML

# Dictionary to map method names to sklearn models
model_mapping = {
    "Logistic Regression": LogisticRegression,
    "Random Forest": RandomForestClassifier,
    "SVM": SVC,
}

## dimension of dataset

In [26]:
print(train_x.shape, train_y.shape)

(3520, 11769) (3520,)


## cek file of config

In [31]:
choosen_method = config['choosen_method']
model_version = config['model_versions']

print(f"Training with {choosen_method}, model version {model_version}")

Training with SVM, model version v1.0.0


In [33]:
method = [x for x in config['methods'] if x['name'] == choosen_method][0]

model_name = method["name"]
model_config = convert_params(method["config"])

model_namespace = f"models/{model_name}/{model_version}"

for param_name, param_value in model_config.items():
    run[f"{model_namespace}/parameters/{param_name}"] = param_value
    
# Instantiate model with specified config
ModelClass = model_mapping[model_name]
model = ModelClass(**model_config)

# Train model
model.fit(train_x, train_y)

 # Save the model to a file
model_filename = f"{model_name}_{model_version}.joblib"
joblib.dump(model, model_filename)

# Log model artifact to Neptune
run[f"{model_namespace}/artifact"].upload(model_filename)

# Predict and evaluate
predicted_y = model.predict(test_x)
report = classification_report(test_y, predicted_y)

# Log classification report to Neptune
run[f"{model_namespace}/classification_report"] = report

print(f"Classification Report for {model_name}:\n{report}\n")

run.stop()

Classification Report for SVM:
              precision    recall  f1-score   support

       anger       0.59      0.80      0.68       220
        fear       0.89      0.56      0.69       130
       happy       0.64      0.60      0.62       204
        love       0.86      0.62      0.72       127
     sadness       0.49      0.53      0.51       200

    accuracy                           0.63       881
   macro avg       0.69      0.62      0.64       881
weighted avg       0.66      0.63      0.64       881


[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 2 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 2 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/test-text/bootcamp-test/e/BOOT-1/metadata
